<a href="https://colab.research.google.com/github/rollmark/AiLearning/blob/master/en_hotels_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 4.4 MB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.4 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 596 kB 74.0 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c8be33fb4cd7c55d76601aad11719fc94bf6cc93a77666663f5a546f77bb3fd3
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd 
import re
import string
from sentence_transformers import SentenceTransformer,util

In [ ]:
model  = SentenceTransformer('all-mpnet-base-v2')  # all-mpnet-base-v2

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# upload file
from google.colab import files
upload = files.upload()

Saving amex_hotel-names-with-cosin-sim.xlsx to amex_hotel-names-with-cosin-sim.xlsx


In [ ]:
%time
query_embedding = model.encode(['How big is London','London has 9,787,426 inhabitants at the 2011 census'], convert_to_tensor=True)
passage_embedding = model.encode(['How big is London','London has 9,787,426 inhabitants at the 2011 census'],convert_to_tensor=True)
query_embedding = query_embedding.to('cuda')
query_embedding = util.normalize_embeddings(query_embedding)
passage_embedding = passage_embedding.to('cuda')
passage_embedding = util.normalize_embeddings(passage_embedding)
sim = util.cos_sim(query_embedding, passage_embedding)
print("Similarity:", sim)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
Similarity: tensor([[1.0000, 0.6286],
        [0.6286, 1.0000]], device='cuda:0')


In [ ]:
df =pd.read_excel("amex-en-hotels.xlsx")

In [ ]:
df_sub = df[['Master_ID','NAME_CN', 'NAME_EN','Sabre Property ID','Sabre Property name']]
df_active = df_sub[df_sub['Sabre Property name'] != 'Inactive Property']
df_active.reindex()
df_active.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162386 entries, 0 to 196479
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Master_ID            162386 non-null  int64 
 1   NAME_CN              162386 non-null  object
 2   NAME_EN              162386 non-null  object
 3   Sabre Property ID    162386 non-null  object
 4   Sabre Property name  162386 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.4+ MB


In [ ]:

import re
import string

def remove_puct(text):
	#print(text)
	trans_table = text.maketrans('', '', string.punctuation)
	text = text.translate(trans_table)
	return text

def process_name(df, text_column):
	df[text_column] = df[text_column].str.strip()
	df[text_column] = df[text_column].str.lower()
	df[text_column] = df[text_column].apply(lambda x: remove_puct(x))
	#df[text_column] = df[text_column].apply(lambda x: remove_bracket(x))
	#df['is_contain_chinese'] = df[text_column].apply(lambda x: isContainsChinese(x))
	return df


In [ ]:
df_active = df_active[df_active['NAME_EN'].astype(str).str.len() >= 3]
df_active = df_active[df_active['Sabre Property name'].astype(str).str.len() >= 3]
df_active.drop_duplicates(subset=['Master_ID', 'Sabre Property ID'], keep='first',inplace=True)
df_active = process_name(df_active,'NAME_EN')
df_active = process_name(df_active,'Sabre Property name')

In [ ]:
master_names = df_active['NAME_EN'].tolist()
sabre_names = df_active['Sabre Property name'].tolist()


In [ ]:

master_embeddings = model.encode(master_names,convert_to_tensor=True,show_progress_bar=True)
master_embeddings = master_embeddings.to('cuda')
master_embeddings = util.normalize_embeddings(master_embeddings)

sabre_embeddings = model.encode(sabre_names,convert_to_tensor=True,show_progress_bar=True)
sabre_embeddings = sabre_embeddings.to('cuda')
sabre_embeddings = util.normalize_embeddings(sabre_embeddings)

Batches:   0%|          | 0/5075 [00:00<?, ?it/s]

Batches:   0%|          | 0/5075 [00:00<?, ?it/s]

In [ ]:
ip1 = master_embeddings[0:5]
ip2 = sabre_embeddings[0:5]

In [ ]:
ip1.shape

torch.Size([5, 768])

In [ ]:
from torch import nn

In [ ]:
cos = nn.CosineSimilarity(dim=1, eps=1e-8)

In [ ]:
opt = cos(ip1,ip2)

In [ ]:
opt

tensor([1.0000, 0.9004, 1.0000, 1.0000, 0.9476], device='cuda:0')

In [ ]:
%time
res = cos(master_embeddings,sabre_embeddings)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [ ]:
res.shape

torch.Size([162372])

In [ ]:
res[:5]

tensor([1.0000, 0.9004, 1.0000, 1.0000, 0.9476], device='cuda:0')

In [ ]:
sims_list = res.tolist()

In [ ]:
sims_list[:5]

[1.0000001192092896,
 0.9003801345825195,
 0.9999999403953552,
 1.0,
 0.9475671052932739]

In [ ]:
df_active['cos_sim'] = sims_list

In [ ]:
df_active.head()

,Master_ID,NAME_CN,NAME_EN,Sabre Property ID,Sabre Property name,cos_sim
0,2030,香港九龙海逸君绰酒店,harbour grand kowloon,18624,harbour grand kowloon,1.000000
1,2548,香港富豪九龙酒店,regal kowloon hotel sterling h,10774,regal kowloon hotel,0.900380
2,2549,香港九龙酒店,the kowloon hotel,9658,the kowloon hotel,1.000000
3,2780,香港帝苑酒店,the royal garden,4760,the royal garden,1.000000
4,40782,香港东方泛达酒店,oriental lander hotel hong ko,257305,oriental lander hotel hong kong,0.947567


In [ ]:
df_active.to_excel('amex_hotel-names-with-cosin-sim.xlsx')

In [ ]:
df_active[df_active['cos_sim'] <=0.9]

,Master_ID,NAME_CN,NAME_EN,Sabre Property ID,Sabre Property name,cos_sim
7,1556,香港湾景国际酒店,the harbourview,51520,the harbourview nearby hong kong exhibition a...,0.629650
9,2036,香港沙田万怡酒店,courtyard sha tin marriott,174778,courtyard hong kong sha tin,0.759863
13,28579,香港MK酒店,hotel mk formerly known as tata,253467,hotel mk formerly known as tatami hampton,0.883316
14,53142,BEST WESTERN PLUS SLIDELL HTL,best western plus slidell htl,58684,best western slidell hotel,0.768968
16,53171,LEMON TREE AMARANTE,lemon tree amarante,81102,lemon tree amarante beach resort goa,0.716338
...,...,...,...,...,...,...
196457,2377833,HOLIDAY INN EXP WELLS,holiday inn exp wells,392317,holiday inn express wellsogunquitkennebunk,0.682881
196468,2377898,SERVICE PLUS INNS AND SUITES GR,service plus inns and suites gr,392437,service plus inns and suites grande prairie,0.802153
196471,2377926,RAMADA ENCORE CHINATOWN KL,ramada encore chinatown kl,392498,ramada encore by wyndham chinatown kuala lumpur,0.875156
196474,2377949,HGI UNIV CAP DISTRICT,hgi univ cap district,392557,hilton garden inn austin university capitol di...,0.375714


In [ ]:
files.download('amex_hotel-names-with-cosin-sim.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#### Train 
train_size = int(len(master_names) * 0.95)
train_size

154253

In [ ]:
from datetime import datetime
from sentence_transformers import losses,evaluation
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
import random

In [ ]:
model_save_path = 'training_amex_en_MultipleNegativesRankingLoss-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
model_save_path

'training_amex_en_MultipleNegativesRankingLoss-2022-07-18_07-39-35'

In [ ]:
num_epochs = 10
train_batch_size = 128


In [ ]:
train_samples = []
train_master_names = master_names[:train_size]
train_sabre_names = sabre_names[:train_size]
for i in range(len(train_master_names)):
    train_samples.append(InputExample(texts=[train_master_names[i],train_sabre_names[i]],label=1))
    train_samples.append(InputExample(texts=[train_sabre_names[i],train_master_names[i]],label=1))

In [ ]:
len(train_samples)

308506

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size = train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
evaluators = []
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []
eva_master_names = master_names[train_size:]
eva_sabre_names = sabre_names[train_size:]
for i in range(len(eva_master_names)):
    dev_sentences1.append(eva_master_names[i])
    dev_sentences2.append(eva_sabre_names[i])
    dev_labels.append(1)

binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, dev_sentences2, dev_labels)
evaluators.append(binary_acc_evaluator)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=binary_acc_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path = model_save_path)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2411 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2411 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2411 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2411 [00:00<?, ?it/s]

In [ ]:
df =pd.read_excel("amex_hotel-names-with-cosin-sim.xlsx")
df = df[df['cos_sim'] >= 0.75]
master_names = df['NAME_EN'].tolist()
sabre_names = df['Sabre Property name'].tolist()

In [ ]:
from datetime import datetime
from sentence_transformers import losses,evaluation
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
import random

In [ ]:
#### Train 
train_size = int(len(master_names) * 0.95)

model_save_path = 'training_amex_hotels_en_MultipleNegativesRankingLoss-latest'+datetime.now().strftime("%Y-%m-%d")
print("Model save path:{}".format(model_save_path))
num_epochs = 10
train_batch_size = 256

Model save path:training_amex_hotels_en_MultipleNegativesRankingLoss-latest2022-07-19


In [ ]:


# build train samples
train_samples = []
train_master_names = master_names[:train_size]
train_sabre_names = sabre_names[:train_size]
print("Train size: {}".format(len(train_master_names)))
for i in range(len(train_master_names)):
    train_samples.append(InputExample(texts=[train_master_names[i],train_sabre_names[i]],label=1))
    train_samples.append(InputExample(texts=[train_sabre_names[i],train_master_names[i]],label=1))

# build train datasets and loss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size = train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

# define and init evaluators
evaluators = []
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []
eva_master_names = master_names[train_size:]
eva_sabre_names = sabre_names[train_size:]
print("Eva samples len:{}".format(len(eva_master_names)))
for i in range(len(eva_master_names)):
    dev_sentences1.append(eva_master_names[i])
    dev_sentences2.append(eva_sabre_names[i])
    dev_labels.append(1)

binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, dev_sentences2, dev_labels)
evaluators.append(binary_acc_evaluator)

Train size: 138793
Eva samples len:7305


In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=binary_acc_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path = model_save_path,
         save_best_model=True,
         show_progress_bar=True)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1085 [00:00<?, ?it/s]

In [ ]:
!zip -q -r training_amex_en_MultipleNegativesRankingLoss_latest.zip training_amex_hotels_en_MultipleNegativesRankingLoss-latest2022-07-19
 
files.download('training_amex_en_MultipleNegativesRankingLoss_latest.zip')